<a href="https://colab.research.google.com/github/SeemGoel/TextClassification/blob/master/BERT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 71kB 7.1MB/s 
     |████████████████████████████████| 1.1MB 26.9MB/s 
     |████████████████████████████████| 890kB 52.2MB/s 
     |████████████████████████████████| 3.0MB 54.2MB/s 
     |████████████████████████████████| 1.1MB 54.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.8-cp36-none-any.whl size=101996 sha256=707c5a428d40da0e867df0b2e1ca6b155037bdef84b7f3414c12bf947d664b84
  Stored in directory: /root/.cache/pip/wheels/27/ec/b3/d12cc8e4daf77846db6543033d3a5642f204c0320b15945647
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=62000df5abf1843c409856346e5c410b28393d390bdcb52fc8fd9573dfa396ec
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
!pip install elasticsearch

     |████████████████████████████████| 327kB 21.1MB/s 


In [ ]:
import pandas as pd
import re
import sentence_transformers
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
!pip install tensorflow

In [ ]:
!pip install tensorflow_hub

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:23<00:00, 17.3MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
import json
with open("drive/My Drive/BERT_Task/elastic.json") as elastic_file:
    ELASTIC_SETTINGS = json.loads(elastic_file.read().strip())

In [ ]:
ELASTIC_SETTINGS

{'cloud_id': 'memory-optimized-deployment:ZWFzdHVzMi5henVyZS5lbGFzdGljLWNsb3VkLmNvbTo5MjQzJGI2MDJiZmQ4OWU5MzQ5NGY4NDYwZmJmOGNlMTg5ZGJkJDYwMzUxZTBiM2QzNjRjNGVhNTYwMGI3YmM2MWUwOWM2',
 'password': 'erw5m4KTWyosH231YMAj2u0X',
 'user': 'elastic'}

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(
    cloud_id='memory-optimized-deployment:ZWFzdHVzMi5henVyZS5lbGFzdGljLWNsb3VkLmNvbTo5MjQzJGI2MDJiZmQ4OWU5MzQ5NGY4NDYwZmJmOGNlMTg5ZGJkJDYwMzUxZTBiM2QzNjRjNGVhNTYwMGI3YmM2MWUwOWM2',
    http_auth=('elastic', 'erw5m4KTWyosH231YMAj2u0X'),
)

In [ ]:
es.get(index="index", id="2")

{'_id': '2',
 '_index': 'index',
 '_primary_term': 1,
 '_seq_no': 3,
 '_source': {'body': 'me 123'},
 '_type': '_doc',
 '_version': 3,
 'found': True}

In [ ]:
from tqdm.notebook import tqdm
import os
import json
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import time
import numpy as np

print(tf.__version__)

2.3.0


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
working_list=[]
df_desc = pd.read_csv('gdrive/My Drive/BERT_Task/Input_Data_domain_full_text.csv')

In [ ]:
df_desc.columns

Index(['Unnamed: 0', '_id', '_score', 'text'], dtype='object')

In [ ]:
df_desc=df_desc.rename(columns={"Unnamed: 0":"i","_id":'company','_score':'score','text':'description'})

In [ ]:
df_desc.isna().sum()

i              0
company        0
score          0
description    0
dtype: int64

In [ ]:
df_desc[21:35]

,i,company,score,description
21,21,zv-nm.com,1.0,"['0', '693 East Ave', 'About', 'About Us', 'Co..."
22,22,zuper.co,1.0,"['#001374', '#116ebf', '#3b5998', '#82b541', '..."
23,23,zuora.com,1.0,"['""Having gone through an ERP upgrade that inc..."
24,24,zumolabs.ai,1.0,"['At Zumo Labs, we generate custom synthetic d..."
25,25,zumasys.com,1.0,"['#1 Workplaces for Giving Back 2016', '(Pick ..."
26,26,zuggand.com,1.0,"['& Compliance', '& Drive Alignment', '& IT Ri..."
27,27,zugang.com.au,1.0,"['+65.94875521 / +61410633849 / +17185214592',..."
28,28,zuerns.com,1.0,['$15 – Now Taking Pre-Orders! Provide: Name |...
29,29,ztscorp.com,1.0,"[', or preparing to negotiate a VBP contract, ..."
30,30,zsolutionz.com,1.0,"['$280.00', '$287.95', '$7.95', '1-Year Regist..."


In [ ]:
df_desc[pd.to_numeric(df_desc['i'], errors='coerce').notnull()]

,i,company,score,description
0,0,zzalpha.com,1.0,"['"" gives a more accurate picture: you LOST 13..."
1,1,zz-tmt.com,1.0,"['1', '2', '3', '4', 'ABS', 'AS', 'About TMT',..."
2,2,zytekautomotive.co.uk,1.0,['(Further information can be read on our Hist...
3,3,zytek.co.uk,1.0,['(Further information can be read on our Hist...
4,4,zytecsystems.co.uk,1.0,"['""Since the incorporation of the company we h..."
...,...,...,...,...
20388,20388,ascoeq.com,1.0,"['(800) 687-0008', '(806) 372-1999', '* Fields..."
20389,20389,aschome.com,1.0,"['(818) 833-0080', '(818) 833-0088', '- And mo..."
20390,20390,ascentwebportal.com,1.0,"['""I\'m incredibly pleased that we have establ..."
20391,20391,ascentvp.com,1.0,"['""Ascent is a proven partner for scaling grea..."


In [ ]:
df_desc.dropna(subset=['i'], inplace=True)

In [ ]:
df_desc

,i,company,score,description
0,0,zzalpha.com,1.0,"['"" gives a more accurate picture: you LOST 13..."
1,1,zz-tmt.com,1.0,"['1', '2', '3', '4', 'ABS', 'AS', 'About TMT',..."
2,2,zytekautomotive.co.uk,1.0,['(Further information can be read on our Hist...
3,3,zytek.co.uk,1.0,['(Further information can be read on our Hist...
4,4,zytecsystems.co.uk,1.0,"['""Since the incorporation of the company we h..."
...,...,...,...,...
20388,20388,ascoeq.com,1.0,"['(800) 687-0008', '(806) 372-1999', '* Fields..."
20389,20389,aschome.com,1.0,"['(818) 833-0080', '(818) 833-0088', '- And mo..."
20390,20390,ascentwebportal.com,1.0,"['""I\'m incredibly pleased that we have establ..."
20391,20391,ascentvp.com,1.0,"['""Ascent is a proven partner for scaling grea..."


In [ ]:
df_desc[df_desc['description'].str.contains('[\u4e00-\u9fff]+')==False]

,i,company,score,description
0,0,zzalpha.com,1.0,"['"" gives a more accurate picture: you LOST 13..."
2,2,zytekautomotive.co.uk,1.0,['(Further information can be read on our Hist...
3,3,zytek.co.uk,1.0,['(Further information can be read on our Hist...
4,4,zytecsystems.co.uk,1.0,"['""Since the incorporation of the company we h..."
5,5,zyquest.us,1.0,"['04/12/2016', '05/06/2016', '12/03/2015', '13..."
...,...,...,...,...
20388,20388,ascoeq.com,1.0,"['(800) 687-0008', '(806) 372-1999', '* Fields..."
20389,20389,aschome.com,1.0,"['(818) 833-0080', '(818) 833-0088', '- And mo..."
20390,20390,ascentwebportal.com,1.0,"['""I\'m incredibly pleased that we have establ..."
20391,20391,ascentvp.com,1.0,"['""Ascent is a proven partner for scaling grea..."


In [ ]:
working_list = df_desc[['company','description']]
working_list.loc[:,('description')]=working_list.loc[:,('description')].replace("/"," ")
Workinglist1=working_list[working_list['description'].str.contains('[\u4e00-\u9fff]+')==False]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [ ]:
Workinglist1

,company,description
0,zzalpha.com,"['"" gives a more accurate picture: you LOST 13..."
2,zytekautomotive.co.uk,['(Further information can be read on our Hist...
3,zytek.co.uk,['(Further information can be read on our Hist...
4,zytecsystems.co.uk,"['""Since the incorporation of the company we h..."
5,zyquest.us,"['04/12/2016', '05/06/2016', '12/03/2015', '13..."
...,...,...
20388,ascoeq.com,"['(800) 687-0008', '(806) 372-1999', '* Fields..."
20389,aschome.com,"['(818) 833-0080', '(818) 833-0088', '- And mo..."
20390,ascentwebportal.com,"['""I\'m incredibly pleased that we have establ..."
20391,ascentvp.com,"['""Ascent is a proven partner for scaling grea..."


In [ ]:
working_list

,company,description
0,zzalpha.com,"['"" gives a more accurate picture: you LOST 13..."
1,zz-tmt.com,"['1', '2', '3', '4', 'ABS', 'AS', 'About TMT',..."
2,zytekautomotive.co.uk,['(Further information can be read on our Hist...
3,zytek.co.uk,['(Further information can be read on our Hist...
4,zytecsystems.co.uk,"['""Since the incorporation of the company we h..."
...,...,...
20388,ascoeq.com,"['(800) 687-0008', '(806) 372-1999', '* Fields..."
20389,aschome.com,"['(818) 833-0080', '(818) 833-0088', '- And mo..."
20390,ascentwebportal.com,"['""I\'m incredibly pleased that we have establ..."
20391,ascentvp.com,"['""Ascent is a proven partner for scaling grea..."


In [ ]:
df_duplicates_removed = pd.DataFrame.drop_duplicates(Workinglist1)
print(df_duplicates_removed)

                     company                                        description
0                zzalpha.com  ['" gives a more accurate picture: you LOST 13...
2      zytekautomotive.co.uk  ['(Further information can be read on our Hist...
3                zytek.co.uk  ['(Further information can be read on our Hist...
4         zytecsystems.co.uk  ['"Since the incorporation of the company we h...
5                 zyquest.us  ['04/12/2016', '05/06/2016', '12/03/2015', '13...
...                      ...                                                ...
20388             ascoeq.com  ['(800) 687-0008', '(806) 372-1999', '* Fields...
20389            aschome.com  ['(818) 833-0080', '(818) 833-0088', '- And mo...
20390    ascentwebportal.com  ['"I\'m incredibly pleased that we have establ...
20391           ascentvp.com  ['"Ascent is a proven partner for scaling grea...
20392       ascentvision.com  ['(AVT) specializes in gyro-stabilized imaging...

[18928 rows x 2 columns]


In [ ]:
df_d_r=df_duplicates_removed[working_list['company'] !='zat.cz']
df_d_r=df_d_r[working_list['company'] !='ytpals.com']
df_d_r=df_d_r[working_list['company'] !='yuantiku.com']
df_d_r=df_d_r[working_list['company'] !='xrspace.io']

df_d_r=df_d_r[working_list['company'] !='worktecht.com']
df_d_r.reindex

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


<bound method DataFrame.reindex of                      company                                        description
0                zzalpha.com  ['" gives a more accurate picture: you LOST 13...
2      zytekautomotive.co.uk  ['(Further information can be read on our Hist...
3                zytek.co.uk  ['(Further information can be read on our Hist...
4         zytecsystems.co.uk  ['"Since the incorporation of the company we h...
5                 zyquest.us  ['04/12/2016', '05/06/2016', '12/03/2015', '13...
...                      ...                                                ...
20388             ascoeq.com  ['(800) 687-0008', '(806) 372-1999', '* Fields...
20389            aschome.com  ['(818) 833-0080', '(818) 833-0088', '- And mo...
20390    ascentwebportal.com  ['"I\'m incredibly pleased that we have establ...
20391           ascentvp.com  ['"Ascent is a proven partner for scaling grea...
20392       ascentvision.com  ['(AVT) specializes in gyro-stabilized imaging...

[189

In [ ]:
df_d_r=df_d_r.drop([df_d_r.index[0] , df_d_r.index[1]])

In [ ]:
df_d_r.reindex

<bound method DataFrame.reindex of                    company                                        description
3              zytek.co.uk  ['(Further information can be read on our Hist...
4       zytecsystems.co.uk  ['"Since the incorporation of the company we h...
5               zyquest.us  ['04/12/2016', '05/06/2016', '12/03/2015', '13...
6                 zypp.app  ['+91 98884890060', '1', '10 Comments', '2', '...
7             zypmedia.com  ['(Equinix)', '1160 Battery St, Ste 235, San F...
...                    ...                                                ...
20388           ascoeq.com  ['(800) 687-0008', '(806) 372-1999', '* Fields...
20389          aschome.com  ['(818) 833-0080', '(818) 833-0088', '- And mo...
20390  ascentwebportal.com  ['"I\'m incredibly pleased that we have establ...
20391         ascentvp.com  ['"Ascent is a proven partner for scaling grea...
20392     ascentvision.com  ['(AVT) specializes in gyro-stabilized imaging...

[18926 rows x 2 columns]>

In [ ]:
final_ds=df_d_r['description'][4]

In [ ]:
final_ds.replace("/\/","")

'[\'"Since the incorporation of the company we have seen quite dramatic changes both in technology and the attitude of companies requiring remote access and support services. This sector has also called for infrastructure and security improvements that has increased the scope of our core business to include for high level languages and more integration with company IT systems. It is common for access and camera security systems to be integrated into an overall control and monitoring system with the inclusion of detailed reports for production efficiencies and traceability."\', \'"Zytec Systems Ltd was created to provide clients with a customer focused solution to their automated electrical engineering needs. We encourage a flexible and cooperative attitude to all our engineers and require sincerity at all times."\', \'2020 Zytec Systems(UK) Ltd :\', \'50 Harvest Way\', \'A Rockwell Control Logix PLC with a Rockwell Factory Talk HMI was utilized for the control of the System. The system

In [ ]:
#final_ds.to_csv('domains_description_3900.csv', columns = ['company','description'])

In [ ]:
s="".join(Workinglist1['description'][4])
def get_chunks(s, maxlength):
    start = 0
    end = 0
    while start + maxlength  < len(s) and end != -1:
        end = s.rfind(" ", start, start + maxlength + 1)
        yield s[start:end]
        start = end +1
    yield s[start:]

chunks = get_chunks(s, 512)

#Make list with line lengths:
_chunk= [n for n in chunks]

In [ ]:
_chunk

['[\'"Since the incorporation of the company we have seen quite dramatic changes both in technology and the attitude of companies requiring remote access and support services. This sector has also called for infrastructure and security improvements that has increased the scope of our core business to include for high level languages and more integration with company IT systems. It is common for access and camera security systems to be integrated into an overall control and monitoring system with the inclusion',
 'of detailed reports for production efficiencies and traceability."\', \'"Zytec Systems Ltd was created to provide clients with a customer focused solution to their automated electrical engineering needs. We encourage a flexible and cooperative attitude to all our engineers and require sincerity at all times."\', \'2020 Zytec Systems(UK) Ltd :\', \'50 Harvest Way\', \'A Rockwell Control Logix PLC with a Rockwell Factory Talk HMI was utilized for the control of the System. The s

In [ ]:
sample_doc = _chunk
#print(sample_doc)
for i in sample_doc:
 # print("new")
  print(i)
  print(len(i))
  embeddings = model.encode(i)
  print(embeddings.shape)

#embeddings = model.encode(sample_doc)
#print(embeddings.shape)

['"Since the incorporation of the company we have seen quite dramatic changes both in technology and the attitude of companies requiring remote access and support services. This sector has also called for infrastructure and security improvements that has increased the scope of our core business to include for high level languages and more integration with company IT systems. It is common for access and camera security systems to be integrated into an overall control and monitoring system with the inclusion
511
(768,)
of detailed reports for production efficiencies and traceability."', '"Zytec Systems Ltd was created to provide clients with a customer focused solution to their automated electrical engineering needs. We encourage a flexible and cooperative attitude to all our engineers and require sincerity at all times."', '2020 Zytec Systems(UK) Ltd :', '50 Harvest Way', 'A Rockwell Control Logix PLC with a Rockwell Factory Talk HMI was utilized for the control of the System. The syste

In [ ]:
#sample_doc = Workinglist1['description'][10]
#print(sample_doc)
#embeddings = model.encode([sample_doc])
#print(embeddings)

In [ ]:
#sample_doc = Workinglist1['description'][20]
#print(sample_doc)
#embeddings =sentence_auto([sample_doc])
#print(embeddings)

In [ ]:
#indexautobert =start_auto_91(3900 indexed documents),start_auto_19 (2600 indexed documents)
ES_INDEX_NAME = "start_auto_19"
ES_INDEX_FILE = "gdrive/My Drive/BERT_Task/articles_index.json"

BATCH_SIZE = 40
SEARCH_SIZE = 100

In [ ]:
## Function to embed text
def embed_text(text):
    vectors = model.encode(text)
    return [vector.tolist() for vector in vectors]

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [ ]:
def sentence_auto(sentences):
#Load AutoModel from huggingface model repository
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")
    model = AutoModel.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")

#Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

#Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])    
    return [vector.tolist() for vector in sentence_embeddings]

In [ ]:
def index_batch(docs):
    print(docs)
    description = [doc for doc in docs]
    description_vectors = embed_text(description)
    #print(description_vectors)
    requests = []
    for i, doc in enumerate(docs):
        print(doc)
        request = "doc"
        request["_op_type"] = "index"
        request["_index"] = ES_INDEX_NAME
        request["description_vector"] = description_vectors[i]
        #request["description_vector"]=[1.0,1.0]
        requests.append(request)
        print(requests)
    helpers.bulk(es, requests)

In [ ]:
## index data in batches
def index_data(description):
   
    with open(ES_INDEX_FILE) as index_file:
        source = index_file.read().strip()
        es.indices.create(index=ES_INDEX_NAME, body=source,request_timeout=900)
    
    docs = []
    count = 0

    for line in description:
        #print (line)
        #doc = json.loads(line)
        docs.append(line)
        count += 1

        if count % BATCH_SIZE == 0:
            index_batch(docs)
            docs = []
            print("Indexed {} documents.".format(count))
            
    if docs:
        index_batch(docs)
        print("Indexed {} documents.".format(count))

    es.indices.refresh(index="ES_INDEX_NAME")
    print("Done indexing.")

In [ ]:
#sample_doc = _chunk
description = [print(doc) for doc in _chunk]
desc_vectors=[]
for doc in _chunk:
  #description.remove(None)
  description_vectors = sentence_auto(doc)
  print(description_vectors)
  print(len(description_vectors))
  desc_vectors.append(description_vectors)
  desc_vectors

['"Since the incorporation of the company we have seen quite dramatic changes both in technology and the attitude of companies requiring remote access and support services. This sector has also called for infrastructure and security improvements that has increased the scope of our core business to include for high level languages and more integration with company IT systems. It is common for access and camera security systems to be integrated into an overall control and monitoring system with the inclusion
of detailed reports for production efficiencies and traceability."', '"Zytec Systems Ltd was created to provide clients with a customer focused solution to their automated electrical engineering needs. We encourage a flexible and cooperative attitude to all our engineers and require sincerity at all times."', '2020 Zytec Systems(UK) Ltd :', '50 Harvest Way', 'A Rockwell Control Logix PLC with a Rockwell Factory Talk HMI was utilized for the control of the System. The system also incl

KeyboardInterrupt: ignored

In [ ]:
desc_vectors

In [ ]:
es

In [ ]:
#indexautobert =start_auto_91(3900 indexed documents),start_auto_19 (2600 indexed documents)
ES_INDEX_NAME = "start_auto_00001"

In [ ]:
index_data(_chunk)

['[\'"Since the incorporation of the company we have seen quite dramatic changes both in technology and the attitude of companies requiring remote access and support services. This sector has also called for infrastructure and security improvements that has increased the scope of our core business to include for high level languages and more integration with company IT systems. It is common for access and camera security systems to be integrated into an overall control and monitoring system with the inclusion', 'of detailed reports for production efficiencies and traceability."\', \'"Zytec Systems Ltd was created to provide clients with a customer focused solution to their automated electrical engineering needs. We encourage a flexible and cooperative attitude to all our engineers and require sincerity at all times."\', \'2020 Zytec Systems(UK) Ltd :\', \'50 Harvest Way\', \'A Rockwell Control Logix PLC with a Rockwell Factory Talk HMI was utilized for the control of the System. The sy

TypeError: ignored

In [ ]:
## Function to handle the query
def run_query_loop(keyword):
        result = handle_query(keyword)
        return result

In [ ]:
from itertools import chain
def handle_query(query):
    embedding_start = time.time()
    query_vector = embed_text([query])[0]
    embedding_time = time.time() - embedding_start
    
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, doc['description_vector']) + 1.0",
                "params": {"query_vector": query_vector}
            }
        }
    }

    search_start = time.time()
    response = es.search(
        index=ES_INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source": {"includes": ["company", "description"]}
        }
    )
    search_time = time.time() - search_start
    results = {}
    a=[]
    print()
    print("{} total hits.".format(response["hits"]["total"]["value"]))
    print("embedding time: {:.2f} ms".format(embedding_time * 1000))
    print("search time: {:.2f} ms".format(search_time * 1000))
    for hit in response["hits"]["hits"]:
      hit=[hit]
      a.append([{**x, **x.pop('_source')} for x in hit])
    df = pd.DataFrame(list(chain.from_iterable(a)))
    return df

In [ ]:
response = es.search(index="start_auto_91", body={"size": 0,"query":{"match_all" : {}  },}  )
print(response)

{'took': 0, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3900, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


In [ ]:
keywords = ['eCommerce', 'insurance','logistics']

In [ ]:
indices_names = []
for elem in es.cat.indices(format="json"):
    indices_names.append( elem['index'] )

dict_index_fields = {}

for index in indices_names: 
    mapping = es.indices.get_mapping(index)
    dict_index_fields[index] = []
    for field in mapping[index]['mappings']['properties']:
        dict_index_fields[index].append(field) 

In [ ]:
#indexautobert = start_auto_be_10, start_auto_b_10,start_auto_19
#ES_INDEX_NAME = "start_auto_91"
#es.indices.delete(index='start_auto_19')

{'acknowledged': True}

In [ ]:
dict_index_fields

In [ ]:
for keyword in keywords:
   result= run_query_loop(keyword)   
   name= f'100k_{keyword}'
   print(name)
   result.to_excel(f'{name}.xlsx')


3900 total hits.
embedding time: 30.90 ms
search time: 53.13 ms
100k_eCommerce

3900 total hits.
embedding time: 44.58 ms
search time: 53.71 ms
100k_insurance

3900 total hits.
embedding time: 93.98 ms
search time: 47.05 ms
100k_logistics


In [ ]:
pip install xlsxwriter

     |████████████████████████████████| 144 kB 6.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
